# Пример раскрашивания черно-белого фильма с помощью Vision Mail.ru

К 9 Мая MAIL.RU запустил онлайн-сервис для реставрации старых фотографий.
https://9may.mail.ru/

В основе сервиса - модель по реставрации изображений, которая делает следующее:

1. находит все дефекты изображения: надломы, потертости, дырки;
2. закрашивает найденные дефекты, основываясь на значениях пикселей вокруг них;
3. раскрашивает изображение.

##  Раскрасим военный фильм с помощью этой модели
Будем действовать по следующему алгоритму:

1. посмотрим как изменится запрос в Vision и отправим изображение
2. посмотрим как изменится ответ
3. напишем новый парсер ответа
4. посмотрим работу с видеофайлами
5. попробуем отправить запросы для каждого кадра
6. запишем рузельтат в видеофайл

Для авторизации в https://mcs.mail.ru/app/services/machinelearning/ нам также понадобится сервисный токен

In [ ]:
token = "ahdps5M7sEXjBpR7cd4U9jP3yFHS7dYwVhrsv76VBC2d1veKm"

1. Будем использовать метод $\textbf{v1/photo/improve}$

In [ ]:
HOST = "https://smarty.mail.ru/api/v1/photo/improve"

Тогда функция запроса будет выглядеть следующим образом (подробнее https://mcs.mail.ru/help/image-processing/improve): 

In [ ]:
def request_backend(img, token):
    files = {}
    name = "file_0"
    files[name] = img
        
    meta = {
            "mode":["improve"],
            "images": [{"name": name}]
           }
                                                              
    data = {'meta': json.dumps(meta)}
    response = requests.post(HOST + "?oauth_provider=mcs&oauth_token={}" \
                      .format(token), data=data, files=files)

    return response

Возьмём черно-белую фотографию военных времен для тестов

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

img = cv2.imread("war.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
plt.figure(figsize=(16,10))

plt.axis("off")
plt.imshow(img)
plt.show()

Отправим её в Vision и получим следующий ответ

In [ ]:
import json
import requests
import pprint as pp

img = open("war.jpg", 'rb')
r = request_backend(img, token)
pp.pprint(r.json())
img.close()

Картинка в ответе возвращается в формате base64, поэтому добавим конвертацию

In [ ]:
def from_base64(base64_data):
    nparr = np.frombuffer(base64.b64decode(base64_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img


def parse_response(response):
    json_data = response.json()
    status = json_data['status']
    
    if status == 200:
        body = json_data['body']
        improve = body['improve'][0]
        
        if improve['status'] != 0:
            print(improve['status'])
            print(improve['error'])
            
        if  'colorized_improved' in improve:
            base64_image = improve['colorized_improved']
        if 'colorized' in improve:
            base64_image = improve['colorized']
            
        img = from_base64(base64_image)
            

    return status, img

Отрисуем результат работы модели

In [ ]:
import base64
import numpy as np

img = open("war.jpg", 'rb')
response = request_backend(img, token)
status, image = parse_response(response)

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(16,10))
plt.axis("off")
plt.imshow(image)
plt.show()

img.close()

Рассмотрим работу с видеофайлом на примере opencv. Выведем первых 5 кадров.

In [ ]:
input_file = "smuglyanka.mp4"

In [ ]:
cap = cv2.VideoCapture(input_file)

num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# рассмотрим первые 6 фреймов
num_frames = min(num_frames, 6)

for iFrame in range(num_frames):
    ret, frame = cap.read()

cap.release()

Отрисуем первые 6 кадров

In [ ]:
def drawFrame(iFrame, image, fig):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig.add_subplot(1, 6, iFrame + 1)
    plt.axis("off")
    plt.imshow(image)
    
cap = cv2.VideoCapture(input_file)

num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames = min(num_frames, 6)

fig = plt.figure(figsize=(16, 16))

for iFrame in range(num_frames):

    ret, frame = cap.read()
    drawFrame(iFrame, frame, fig)

plt.show()
cap.release()

Отправим первые 6 кадров в запросе к Vision

In [ ]:
cap = cv2.VideoCapture(input_file)

num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# рассмотрим первые 6 фреймов
num_frames = min(num_frames, 6)

fig = plt.figure(figsize=(16, 16))

for iFrame in range(num_frames):
    ret, frame = cap.read()
    
    buf = cv2.imencode('.jpg', frame)[1].tostring()
    
    response = request_backend(buf, token)
    status, image = parse_response(response)

    if status != 200:
        print(status)
        continue
    
    drawFrame(iFrame, image, fig)

cap.release()
plt.show()

Из полученных кадров сделаем видеофайл

In [ ]:
output_file = "output.avi"

In [ ]:
from tqdm import tqdm_notebook

cap = cv2.VideoCapture(input_file)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter(output_file,
                      cv2.VideoWriter_fourcc('M','J','P','G'),
                      fps,
                      (frame_width, frame_height))

for iFrame in tqdm_notebook(range(num_frames)):
    ret, frame = cap.read()
    
    if iFrame < 3000:
        continue
    
    if frame is not None:
        buf = cv2.imencode('.jpg', frame)[1].tostring()
        response = request_backend(buf, token)
        status, image = parse_response(response)
        
        if status != 200:
            print(status)
            continue
        
        out.write(image)
        
        # пока, чтобы много денег не тратить!
        if iFrame > 3010: #3800
            break

cap.release()
out.release()